# Building the Transformer from Scratch

In this notebook, we'll be implementing the famous Transformer architecture from scratch.

The code is based off of the following repos/blog posts:

- [attention-is-all-you-need-pytorch](https://github.com/jadore801120/attention-is-all-you-need-pytorch)
- [pytorch-pretrained-BERT](https://github.com/huggingface/pytorch-pretrained-BERT)
- [The Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention.html) 

Thanks so much to their authors!

In [1]:
import torch
import torch.nn as nn
import numpy as np

One of the keys to understanding how any model works is understanding how the shapes of the tensors change during the processing of each part. We'll be using the logging module to output debugging information to help our understanding.

In [66]:
import logging
logger = logging.getLogger("tensor_shapes")
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
# if you want the model to continuously print tensor shapes, set to DEBUG!
logger.setLevel(1)

In [60]:
import inspect
def getclass():
    stack = inspect.stack()
    return stack[3][0].f_locals["self"].__class__

# A helper function to check how tensor sizes change
def log_size(tsr: torch.Tensor, name: str):
    cls = getclass()
    logger.log(level=cls.level, msg=f"[{cls.__name__}] {name} size={tsr.shape}")

We'll use logging levels to control the modules we receive output from. The lower the logging level, the more tensor information you'll get. Feel free to play around!

In [61]:
from enum import IntEnum
# Control how much debugging output we want
class TensorLoggingLevels(IntEnum):
    attention = 1
    attention_head = 2
    multihead_attention_block = 3
    enc_dec_block = 4
    enc_dec = 5

We'll be using an enum to refer to dimensions whenever possible to improve readability.

In [5]:
class Dim(IntEnum):
    batch = 0
    seq = 1
    feature = 2

# Components

### Scaled dot product attention

The Transformer is an attention-based architecture. The attention used in the Transformer is the scaled dot product attention, represented by the following formula.

##### torch.bmm: https://pytorch.org/docs/stable/torch.html#torch.bmm
Performs a batch matrix-matrix product of matrices stored in input and mat2.

*input and mat2 must be 3-D tensors each containing the same number of matrices ie. (batch).*


If input is a(b×n×m) tensor, mat2 is a (b×m×p) tensor, out will be a (b×n×p) tensor.

##### torch.transpose: https://pytorch.org/docs/stable/torch.html#torch.transpose
torch.transpose(input, dim0, dim1) → Tensor

Returns a tensor that is a transposed version of input. The given dimensions dim0 and dim1 are swapped.

##### torch.sum https://pytorch.org/docs/stable/torch.html#torch.sum
Returns the sum of each row of the input tensor in the given dimension dim. If dim is a list of dimensions, reduce over all of them.

If keepdim is True, the output tensor is of the same size as input

$$ \textrm{Attention}(Q, K, V) = \textrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V $$

![images](./assets/scaled-dot-product-attention-steps.png)

In [85]:
import math

class ScaledDotProductAttention(nn.Module):
    level = TensorLoggingLevels.attention # Logging level: 
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        # q,k,v size = 5,10,20 (Batch, seq, feature)
        
        # Check if key and query size are the same
        d_k = k.size(-1) # get the size of the key
        assert q.size(-1) == d_k

        # STEP 1: Calculate score
        # compute the dot product between queries and keys for each batch and position in the sequence
        k = k.transpose(Dim.seq, Dim.feature) #1, 2 #k.size: 5,20,10
        score_attn = torch.bmm(q, k) # (Batch, Seq, Seq)
        # we get an attention score between each position in the sequence for each batch

        
        # STEP 2: Divide by sqrt(Dk)
        # scale the dot products by the dimensionality. Normalize the weights across the sequence dimension
        score_attn = score_attn / math.sqrt(d_k)
        # (Note that since we transposed, the sequence and feature dimensions are switched)
        
        # STEP 3: Mask Optional
        # fill attention weights with 0s where padded
        if mask is not None: score_attn = score_attn.masked_fill(mask, 0)

        # STEP 4: Softmax    
        score_attn = torch.exp(score_attn)
        log_size(score_attn, "attention weight") # (Batch, Seq, Seq)
        score_attn = score_attn / score_attn.sum(dim=-1, keepdim=True)
        
        score_attn = self.dropout(score_attn)
        
        
        # STEP 5: Matmul with value matrix
        output = torch.bmm(score_attn, v) # (Batch, Seq, Feature)
        
        log_size(output, "attention output size") # (Batch, Seq, Seq)
        return output

In [86]:
attn = ScaledDotProductAttention()

In [87]:
q = torch.rand(5, 10, 20)
k = torch.rand(5, 10, 20)
v = torch.rand(5, 10, 20)

In [88]:
attn(q, k, v)

[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])


tensor([[[0.7417, 0.6113, 0.5676, 0.5910, 0.5614, 0.6212, 0.5809, 0.5165,
          0.4292, 0.5281, 0.3755, 0.7405, 0.7570, 0.6680, 0.5172, 0.6548,
          0.5355, 0.5850, 0.5552, 0.4886],
         [0.7447, 0.6208, 0.5685, 0.6119, 0.5555, 0.6326, 0.5775, 0.5204,
          0.4490, 0.5376, 0.3877, 0.7308, 0.7370, 0.6580, 0.5193, 0.6560,
          0.5501, 0.6061, 0.5813, 0.5215],
         [0.6723, 0.5889, 0.4827, 0.5745, 0.4901, 0.5346, 0.5524, 0.4718,
          0.3610, 0.4852, 0.3613, 0.6532, 0.6971, 0.6579, 0.4699, 0.6321,
          0.5309, 0.5361, 0.4772, 0.4326],
         [0.6854, 0.5194, 0.5535, 0.5398, 0.5483, 0.6098, 0.5069, 0.4946,
          0.4210, 0.4893, 0.3324, 0.6358, 0.6551, 0.5674, 0.5028, 0.5502,
          0.4830, 0.5634, 0.4816, 0.5012],
         [0.7511, 0.6158, 0.5658, 0.5963, 0.5664, 0.6350, 0.5755, 0.5155,
          0.4336, 0.5376, 0.3873, 0.7479, 0.7472, 0.6654, 0.5167, 0.6549,
          0.5400, 0.5954, 0.5667, 0.5085],
         [0.6320, 0.5101, 0.4774, 0.4922, 0.5

### Multi-Head Attention

Now, we turn to the core component in the Transformer architecture: the multi-head attention block. This block applies linear transformations to the input, then applies scaled dot product attention.

![image](https://i2.wp.com/mlexplained.com/wp-content/uploads/2017/12/multi_head_attention.png?zoom=2&resize=224%2C293)

In [10]:
class AttentionHead(nn.Module):
    level = TensorLoggingLevels.attention_head
    def __init__(self, d_model, d_feature, dropout=0.1):
        super().__init__()
        # We will assume the queries, keys, and values all have the same feature size
        self.attn = ScaledDotProductAttention(dropout)
        self.query_tfm = nn.Linear(d_model, d_feature)
        self.key_tfm = nn.Linear(d_model, d_feature)
        self.value_tfm = nn.Linear(d_model, d_feature)

    def forward(self, queries, keys, values, mask=None):
        Q = self.query_tfm(queries) # (Batch, Seq, Feature)
        K = self.key_tfm(keys) # (Batch, Seq, Feature)
        V = self.value_tfm(values) # (Batch, Seq, Feature)
        log_size(Q, "queries, keys, vals")
        # compute multiple attention weighted sums
        x = self.attn(Q, K, V)
        return x

In [11]:
attn_head = AttentionHead(20, 20)
attn_head(q, k, v)

[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])


tensor([[[ 2.5142e-01, -1.5114e-01,  1.8654e-01, -1.5269e-02, -2.0554e-02,
          -2.6997e-01,  3.6803e-01,  3.2500e-01, -4.4870e-01,  2.9316e-01,
           6.6881e-01, -4.7138e-01,  1.1629e-02,  4.9833e-02, -1.0520e-01,
           3.2230e-01, -8.6525e-02,  2.3126e-01,  9.3712e-03,  4.5779e-01],
         [ 2.0075e-01, -9.7678e-02,  8.3727e-02,  4.0386e-02, -6.9687e-02,
          -2.2826e-01,  3.1940e-01,  2.7730e-01, -3.1985e-01,  1.6904e-01,
           4.8410e-01, -3.7480e-01, -1.6219e-02,  5.6603e-02, -1.2206e-01,
           2.7324e-01, -4.3632e-02,  1.8233e-01, -4.7860e-03,  3.5455e-01],
         [ 2.2820e-01, -1.5924e-01,  1.9143e-01,  3.2332e-03,  3.3688e-02,
          -2.6777e-01,  3.0001e-01,  3.0079e-01, -3.2704e-01,  2.6293e-01,
           6.1354e-01, -3.5711e-01,  1.8396e-02,  8.2449e-02, -1.1521e-01,
           2.9854e-01, -6.6571e-02,  1.9163e-01,  1.8653e-02,  3.7795e-01],
         [ 2.8846e-01, -2.0000e-01,  1.8258e-01, -6.9057e-03, -1.8119e-02,
          -3.1183e-01,

The multi-head attention block simply applies multiple attention heads, then concatenates the outputs and applies a single linear projection.

In [12]:
# We'll supress logging from the scaled dot product attention now
logger.setLevel(TensorLoggingLevels.attention_head)

##### repeat() : Repeats this tensor along the specified dimensions. https://pytorch.org/docs/stable/tensors.html#torch.Tensor.repeat

In [95]:
class MultiHeadAttention(nn.Module):
    level = TensorLoggingLevels.multihead_attention_block
    def __init__(self, d_model, d_feature, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.d_feature = d_feature
        self.n_heads = n_heads
        # in practice, d_model == d_feature * n_heads
        assert d_model == d_feature * n_heads

        # Note that this is very inefficient:
        # I am merely implementing the heads separately because it is 
        # easier to understand this way
        self.attn_heads = nn.ModuleList([
            AttentionHead(d_model, d_feature, dropout) for _ in range(n_heads)
        ])
        self.projection = nn.Linear(d_feature * n_heads, d_model) 
    
    def forward(self, queries, keys, values, mask=None):
        log_size(queries, "Input queries")
        x = [attn(queries, keys, values, mask=mask) # (Batch, Seq, Feature)
             for i, attn in enumerate(self.attn_heads)]
        log_size(x[0], "output of single head")
        
        # reconcatenate
        x = torch.cat(x, dim=Dim.feature) # (Batch, Seq, D_Feature * n_heads)
        log_size(x, "concatenated output")
        
        # Final linear operation
        x = self.projection(x) # (Batch, Seq, D_Model)
        log_size(x, "projected output")
        return x

In [97]:
#d_model & d_feature are dimension for our linear operation. Ie. nn.Linear(d_model, d_feature)
d_model = 20 * 8
d_feature = 20
n_heads = 8

heads = MultiHeadAttention(d_model, d_feature, n_heads)
heads(q.repeat(1, 1, 8), 
      k.repeat(1, 1, 8), 
      v.repeat(1, 1, 8))

#q.repeat(1,1,8).size() => torch.Size([5, 10, 160])
#k.repeat(1,1,8).size() => torch.Size([5, 10, 160])
#v.repeat(1,1,8).size() => torch.Size([5, 10, 160])

[MultiHeadAttention] Input queries size=torch.Size([5, 10, 160])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 160])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 160])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queri

tensor([[[-0.1593, -0.1515,  0.1382,  ..., -0.2638, -0.1695,  0.1944],
         [-0.1466, -0.1222,  0.1361,  ..., -0.1853, -0.1153,  0.1861],
         [-0.1503, -0.1859,  0.0968,  ..., -0.2396, -0.1392,  0.2225],
         ...,
         [-0.1632, -0.1102,  0.0614,  ..., -0.2435, -0.1446,  0.1956],
         [-0.1915, -0.1331,  0.1185,  ..., -0.2642, -0.1337,  0.1914],
         [-0.1897, -0.1741,  0.0944,  ..., -0.2772, -0.1739,  0.1774]],

        [[-0.1200, -0.1194,  0.1329,  ..., -0.1804, -0.0830,  0.1663],
         [-0.1086, -0.1202,  0.1108,  ..., -0.1850, -0.0800,  0.1741],
         [-0.1288, -0.1390,  0.1101,  ..., -0.1954, -0.1033,  0.1758],
         ...,
         [-0.1141, -0.0744,  0.0339,  ..., -0.2044, -0.1073,  0.1878],
         [-0.0914, -0.1333,  0.1424,  ..., -0.1461, -0.0915,  0.1575],
         [-0.1207, -0.1414,  0.1184,  ..., -0.1859, -0.0994,  0.1658]],

        [[-0.1660, -0.1548,  0.1685,  ..., -0.1551, -0.0853,  0.1674],
         [-0.1831, -0.1423,  0.1554,  ..., -0

### The Encoder

With these core components in place, implementing the encoder is pretty easy.

![image](./assets/encoder-steps.png)

The encoder consists of the following components:
- A multi-head attention block
- A simple feedforward neural network

These components are connected using residual connections and layer normalization

In [15]:
# We'll supress logging from the individual attention heads
logger.setLevel(TensorLoggingLevels.multihead_attention_block)

Layer normalization is similar to batch normalization, but normalizes across the feature dimension instead of the batch dimension.

![image](https://i1.wp.com/mlexplained.com/wp-content/uploads/2018/01/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2018-01-11-11.48.12.png?w=1500)

In [104]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-8):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

The encoder just stacks these together

In [105]:
class EncoderBlock(nn.Module):
    level = TensorLoggingLevels.enc_dec_block
    def __init__(self, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        
        self.attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        
        self.layer_norm1 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
        self.position_wise_feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )
        
        self.layer_norm2 = LayerNorm(d_model)
        
    def forward(self, x, mask=None):
        log_size(x, "Encoder block input")
        
        # STEP 1
        att = self.attn_head(x, x, x, mask=mask)
        log_size(x, "Attention output")
        
        # STEP 2
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm1(att))
        
        # STEP 3
        # Apply position-wise feedforward network
        pos = self.position_wise_feed_forward(x)
        log_size(x, "Feedforward output")
        
        # STEP 4
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm2(pos))
        log_size(x, "Encoder size output")
        return x

In [106]:
enc = EncoderBlock()

In [107]:
enc(torch.rand(5, 10, 512))

[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 64])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 64])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 64])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 64])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 64])
[ScaledDotProductAttention] attenti

tensor([[[ 0.0108,  1.0342,  0.5324,  ..., -0.9076,  1.1966, -1.9913],
         [ 1.9889,  1.0661, -0.2731,  ...,  0.1506,  0.7549, -1.8243],
         [-0.1332,  1.1098, -0.8688,  ..., -0.7683,  1.2096, -2.3394],
         ...,
         [ 1.3884,  1.6255,  0.4349,  ..., -0.1239,  1.5403, -1.1731],
         [ 0.1003,  1.7683, -0.3073,  ..., -0.3618,  1.2750,  0.2557],
         [-0.1606,  1.5961,  0.3885,  ..., -0.0957,  0.9493, -2.0467]],

        [[-0.0468,  2.0994,  0.4432,  ..., -0.0439,  1.3845, -2.3946],
         [ 0.9454,  1.8005,  0.5436,  ...,  0.8469,  0.5925, -1.7992],
         [ 0.2430,  1.5768,  0.2729,  ..., -0.8551,  1.6647, -1.8886],
         ...,
         [-0.5455,  1.7989, -0.4916,  ...,  1.5479, -0.0375, -1.1887],
         [ 0.8584,  3.1980,  0.5049,  ..., -0.3918,  0.5090, -1.8666],
         [ 0.2035,  2.3547, -0.6481,  ..., -0.1720,  1.6655, -1.4895]],

        [[ 0.3635,  1.3213,  0.4483,  ..., -0.5096,  1.0134, -1.8842],
         [-0.8481,  0.9727,  0.2459,  ..., -0

The encoder consists of 6 consecutive encoder blocks, so can simply be implemented like the following

In [108]:
class TransformerEncoder(nn.Module):
    level = TensorLoggingLevels.enc_dec
    def __init__(self, n_blocks=6, d_model=512,
                 n_heads=8, d_ff=2048, dropout=0.1):
        super().__init__()
        self.encoders = nn.ModuleList([
            EncoderBlock(d_model=d_model, d_feature=d_model // n_heads,
                         d_ff=d_ff, dropout=dropout)
            for _ in range(n_blocks)
        ])
    
    def forward(self, x: torch.FloatTensor, mask=None):
        for encoder in self.encoders:
            x = encoder(x)
        return x

### The Decoder

The decoder is mostly the same as the encoder. There's just one additional multi-head attention block that takes the target sentence as input.

![image](./assets/decoder-steps.png)

The keys and values are the outputs of the encoder, and the queries are the outputs of the multi-head attention over the target entence embeddings.

In [109]:
class DecoderBlock(nn.Module):
    level = TensorLoggingLevels.enc_dec_block
    def __init__(self, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        self.masked_attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        
        self.attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        
        self.position_wise_feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )

        self.layer_norm1 = LayerNorm(d_model)
        self.layer_norm2 = LayerNorm(d_model)
        self.layer_norm3 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_out, 
                src_mask=None, tgt_mask=None):
        # Step 1
        # Apply attention to inputs
        att = self.masked_attn_head(x, x, x, mask=src_mask)
        x = x + self.dropout(self.layer_norm1(att))
        
        # Step 2 
        # Apply attention to the encoder outputs and outputs of the previous layer
        att = self.attn_head(queries=x, keys=enc_out, values=enc_out, mask=tgt_mask)
        x = x + self.dropout(self.layer_norm2(att))
        
        # Step 3
        # Apply position-wise feedforward network
        pos = self.position_wise_feed_forward(x)
        x = x + self.dropout(self.layer_norm2(pos))
        return x

In [22]:
dec = DecoderBlock()
dec(torch.rand(5, 10, 512), enc(torch.rand(5, 10, 512)))

[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[EncoderBlock] Attention output size=torch.Size([5, 10, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 10, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Siz

tensor([[[ 0.0089, -0.1621, -1.7238,  ..., -0.2977, -0.0777,  0.6444],
         [-0.6819,  1.0120, -2.2252,  ..., -0.7179, -0.4535,  0.4894],
         [-0.2763, -1.2217, -0.4251,  ..., -1.1588,  0.5523,  0.2806],
         ...,
         [-0.9637,  0.0112, -1.8797,  ..., -1.4102, -0.3725,  1.9287],
         [-1.0610,  0.0212, -0.1304,  ..., -0.9032, -0.1420, -0.0251],
         [ 0.0933,  0.3426, -0.6407,  ..., -0.3894,  0.3314,  1.4376]],

        [[-0.5598,  0.2414, -1.2082,  ..., -1.3604,  0.7519,  0.3092],
         [-0.1707, -0.4689, -2.7463,  ..., -1.5195, -0.6086,  0.0534],
         [-0.0418,  0.4068, -1.9493,  ..., -1.8899, -0.0944,  0.9726],
         ...,
         [ 0.0187, -0.0970, -1.2934,  ..., -1.6659, -0.3797, -1.0419],
         [-0.6441, -1.6213, -2.8262,  ..., -1.1339,  0.4981,  0.1986],
         [-0.3408,  0.8055, -2.1190,  ..., -1.1125, -0.3273,  0.6981]],

        [[ 0.4155,  2.4197, -1.8789,  ..., -1.0938,  1.2254,  1.3786],
         [-1.2558,  0.5798, -0.9764,  ..., -0

Again, the decoder is just a stack of the underlying block so is simple to implement.

In [110]:
class TransformerDecoder(nn.Module):
    level = TensorLoggingLevels.enc_dec
    def __init__(self, n_blocks=6, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        self.position_embedding = PositionalEmbedding(d_model)
        self.decoders = nn.ModuleList([
            DecoderBlock(d_model=d_model, d_feature=d_model // n_heads,
                         d_ff=d_ff, dropout=dropout)
            for _ in range(n_blocks)
        ])
        
    def forward(self, x: torch.FloatTensor, 
                enc_out: torch.FloatTensor, 
                src_mask=None, tgt_mask=None):
        for decoder in self.decoders:
            x = decoder(x, enc_out, src_mask=src_mask, tgt_mask=tgt_mask)
        return x

### Positional Embeddings

Attention blocks are just simple matrix multiplications: therefore they don't have any notion of order! The Transformer explicitly adds positional information via the positional embeddings.

In [24]:
class PositionalEmbedding(nn.Module):
    level = 1
    def __init__(self, d_model, max_len=512):
        super().__init__()        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.weight = nn.Parameter(pe, requires_grad=False)
        
    def forward(self, x):
        return self.weight[:, :x.size(1), :] # (1, Seq, Feature)

In [25]:
class WordPositionEmbedding(nn.Module):
    level = 1
    def __init__(self, vocab_size, d_model=512):
        super().__init__()
        self.word_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = PositionalEmbedding(d_model)
        
    def forward(self, x: torch.LongTensor, mask=None) -> torch.FloatTensor:
        return self.word_embedding(x) + self.position_embedding(x)

In [26]:
emb = WordPositionEmbedding(1000)
encoder = TransformerEncoder()

In [27]:
encoder(emb(torch.randint(1000, (5, 30))))

[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 30, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 30, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 30, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 30, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 30, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 30, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[

tensor([[[ 4.1055, -0.8502, -4.9186,  ..., -1.2713, -4.2459,  0.2078],
         [ 2.6354, -1.2489, -1.0282,  ...,  3.3799, -1.5298,  5.0705],
         [-0.4808, -4.2213,  1.0662,  ...,  1.8674, -3.0999,  3.9541],
         ...,
         [-1.8011, -0.2372, -2.2997,  ...,  0.0451, -3.3087, 10.8904],
         [ 3.9191, -1.6762, -3.7252,  ...,  5.9903, -0.6455,  3.0996],
         [ 0.4039, -1.9685, -2.4440,  ..., -0.8608, -0.1490,  6.7031]],

        [[ 1.5821,  3.1465,  0.7618,  ..., -0.3407, -0.1425,  3.2112],
         [ 1.6676, -1.2685,  2.1302,  ...,  1.1090, -3.4212,  3.5214],
         [ 3.1787,  2.5393,  3.9663,  ..., -3.6904, -2.7751,  2.0892],
         ...,
         [ 2.4655, -0.7829,  0.0596,  ..., -2.2599,  0.9266,  6.1969],
         [ 0.4926,  2.0922,  3.7247,  ..., -3.8212, -0.7574,  2.4279],
         [ 1.6447,  2.8645, -0.9687,  ..., -1.5437, -0.4660,  4.8466]],

        [[ 0.4819,  4.5232,  1.0103,  ..., -8.8720, -3.7049,  3.7693],
         [-0.3196,  2.4813,  5.1187,  ..., -6

### Putting it All Together

Let's put everything together now.

![image](https://camo.githubusercontent.com/88e8f36ce61dedfd2491885b8df2f68c4d1f92f5/687474703a2f2f696d6775722e636f6d2f316b72463252362e706e67)

In [28]:
# We'll supress logging from the scaled dot product attention now
logger.setLevel(TensorLoggingLevels.enc_dec_block)

In [29]:
emb = WordPositionEmbedding(1000)
encoder = TransformerEncoder()
decoder = TransformerDecoder()

In [30]:
src_ids = torch.randint(1000, (5, 30))
tgt_ids = torch.randint(1000, (5, 30))
x = encoder(emb(src_ids))
decoder(emb(tgt_ids), x)

[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=t

tensor([[[  2.2477,   3.3096,  -2.8686,  ...,  -3.2644,  -2.4669,   6.1730],
         [ -1.8250,  -4.9711,  -2.4306,  ...,  -2.9770,  -3.0167,   1.3833],
         [  3.5309,   2.0060,  -3.2075,  ...,   0.1359,  -4.5663,  -1.5324],
         ...,
         [  7.0745,  -1.9728,   1.4077,  ...,   1.3778,  -0.3159,   1.5097],
         [  5.8074,  -4.9769,   1.1956,  ...,  -0.4884,   0.6054,   2.2616],
         [ -0.5769,   3.2016,  -0.5948,  ...,  -1.7086,  -3.5718,   3.0980]],

        [[  4.1633,   1.9668,  -0.4448,  ...,  -0.5528,  -7.7785,  -0.5628],
         [  6.3603,   4.5621,   1.4731,  ...,   2.1059,  -9.3620,  -4.9059],
         [  1.9835,  -2.7217,  -1.0474,  ...,   1.4569,  -7.1355,   0.1433],
         ...,
         [  2.3934,   1.2819,   1.9249,  ...,  -3.7213,  -9.2985,  -5.8557],
         [ -0.0274,   0.3156,  -0.6774,  ...,   1.5031, -10.2572,  -4.0842],
         [ -2.2030,   4.4538,   0.1307,  ...,  -1.5143, -10.7648,  -1.3035]],

        [[  3.4523,   8.5233,  -1.9825,  ...